# Import Necessary Packages 

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

# Import JSON Data 

In [2]:
# Load JSON: json_data
json_data=pd.read_json("world_bank_projects.json")
df=pd.DataFrame(json_data)

# Quick Inspection of the Data

In [3]:
# Print first two row of the data frame 
print(df. head(2))

                                    _id  approvalfy board_approval_month  \
0  {'$oid': '52b213b38594d8a2be17c780'}        1999             November   
1  {'$oid': '52b213b38594d8a2be17c781'}        2015             November   

      boardapprovaldate                                 borrower  \
0  2013-11-12T00:00:00Z  FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA   
1  2013-11-04T00:00:00Z                    GOVERNMENT OF TUNISIA   

            closingdate                              country_namecode  \
0  2018-07-07T00:00:00Z  Federal Democratic Republic of Ethiopia!$!ET   
1                   NaN                      Republic of Tunisia!$!TN   

  countrycode                              countryname countryshortname  \
0          ET  Federal Democratic Republic of Ethiopia         Ethiopia   
1          TN                      Republic of Tunisia          Tunisia   

                         ...                           sectorcode source  \
0                        ...                

In [4]:
# lets check the column names and datatype in the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

It seems that 'country_namecode','countrycode','countryname', 'countryshortname' have the same information.

In [5]:
df_country=df[['country_namecode','countrycode','countryname', 'countryshortname']]
df_country.head(5)

,country_namecode,countrycode,countryname,countryshortname
0,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia
1,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia
2,Tuvalu!$!TV,TV,Tuvalu,Tuvalu
3,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of"
4,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho


So we can keep one column ('countryshortname') and delete the other three from df.

In [6]:
df.drop(['country_namecode','countrycode','countryname'],axis=1);

# Question 1
> *Find 10 countries with most projects*

In [7]:
df['countryshortname'].value_counts()[0:10]

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Bangladesh            12
Nepal                 12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64

# Question 2
> *Find the top 10 major project themes (using column 'mjtheme_namecode').*

df['mjtheme_namecode'] was listed as an object when we checked with df.info(). Lets check the 
first few entries of the column to identify the data type

In [8]:
df['mjtheme_namecode'].head()

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

It looks like each cell contains a list of dictionaries (i.e. nested elements). Lets use json_normalize 
to create table from each nested element and append them to form a dataframe with all project codes

In [9]:
data_project_theme=pd.DataFrame()
for row in df['mjtheme_namecode']:
    data=row
    data_project_theme=data_project_theme.append(json_normalize(data))

In [10]:
 data_project_theme.head(5)

,code,name
0,8,Human development
1,11,
0,1,Economic management
1,6,Social protection and risk management
0,5,Trade and integration


In [11]:
# So the top 10 major project teams are -
print(data_project_theme['code'].value_counts()[0:10])

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64


# Question 3 

> *In 2. above you will notice that some entries have only the code and the name is missing.* <br>
> *Create a dataframe with the missing names filled in.*



In [12]:
# Lets find the number of entries in 'name' that does not have any entries
print(data_project_theme['name'].value_counts()[''])

122


- Lets try to make a dictionary for all the project codes and the project names associated with them. This will allow to look at > the code for a row with missing project name in the dataframe and then fill in the entry for name  using the dictionary. 
- To make the dictionary, lets make a dataframe first containing all the unique codes and the project_theme.b

In [13]:
# Lets find the number of unique rows
code_name = data_project_theme[data_project_theme['name'] != ''].drop_duplicates()
code_name

,code,name
0,8,Human development
0,1,Economic management
1,6,Social protection and risk management
0,5,Trade and integration
1,2,Public sector governance
2,11,Environment and natural resources management
0,7,Social dev/gender/inclusion
1,4,Financial and private sector development
0,10,Rural development
0,9,Urban development


- Lets convert the dataframe to a dictionary

In [14]:
code_name_dict=dict(zip(code_name.code,code_name.name))
code_name_dict['8']

'Human development'

- Now lets check the dataframe 'data_project_theme' and replace the missing values (i.e. '') with correct project names.

In [15]:
for index,row in data_project_theme.iterrows():
    if row['name'] == '':
        Fill = code_name_dict[str(row['code'])]
        row['name'] = Fill

In [16]:
# Lets take a look at the updated dataframe
data_project_theme.head(5)  

,code,name
0,8,Human development
1,11,Environment and natural resources management
0,1,Economic management
1,6,Social protection and risk management
0,5,Trade and integration
